## Geospatial Data Scientist Census Activity
**Author:** Nicole Pepper

**Date:** 11/21/2025

**GitHub Repo:** https://github.com/nicolelpepper/af-census-activity

### Set Up Workspace

In [1]:
# ---- Load libraries ----
import pandas as pd
import numpy as np
import geopandas as gpd
import rasterio
import re

### Load Project Data

In [2]:
# ---- Read in project data ----

# american community survey data
acs = pd.read_csv("data/raw/ACSDP5Y2023.DP05_2025-11-17T173744/ACSDP5Y2023.DP05-Data.csv")

# landcover data
landcover = rasterio.open("data/raw/landcover/landcover_conus.tif")

# schools data
private_schools = gpd.read_file("data/raw/schools/EDGE_GEOCODE_PRIVATESCH_2122/EDGE_GEOCODE_PRIVATESCH_2122.shp")
public_schools = gpd.read_file("data/raw/schools/EDGE_GEOCODE_PUBLICSCH_2324/EDGE_GEOCODE_PUBLICSCH_2324.shp")

# census tracts data for 2025 (choose a state)
florida_tracts = gpd.read_file("data/raw/tracts/12/tl_2025_12_tract.shp")
# oregon_tracts = gpd.read_file("data/raw/tracts/41/tl_2025_41_tract.shp")
# pennsylvania_tracts = gpd.read_file("data/raw/tracts/42/tl_2025_42_tract.shp")

/var/folders/f1/_qnbjphx7mvb9p2c19t_s1540000gn/T/ipykernel_54307/2079468737.py:4: DtypeWarning: Columns (2,3,4,5,6,7,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,58,59,60,61,62,63,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,238,246,254,324,338,372) have mixed types. Specify dtype option on import or set low_memory=False.
  acs = pd.read_csv("data/raw/ACSDP5Y2023.DP05_2025-11-17T173744/ACSDP5Y2023.DP05-Data.csv")


### Clean Data

##### Prep American Community Survey (acs) Data

In [3]:
# --- Explore  & prep american community survey data ----

# Check column names
acs.columns

# Get snapshot of data
acs.head

# Save 1st row as metadata for columns (bc there are two header rows)
metadata_row = acs.iloc[0]
col_metadata = metadata_row.to_dict()
print(col_metadata)

# Drop extra header and reset index
acs = acs.drop(index=0).reset_index(drop=True)
acs.head

# ---- Select demographic data for Florida ----

# Select primary id fields and 2 demographic statistics of choice: 0037PE (percent white) and 0075PE (percent hispanic)
acs = acs[["GEO_ID", "NAME", "DP05_0001E","DP05_0037PE", "DP05_0075PE"]] 

# filter to Florida
fl_acs = acs[acs["GEO_ID"].str.contains("1400000US12")]

# ---- Clean errors in population statistics columns ----

# Select population statistics columns
pop_cols = ["DP05_0001E","DP05_0037PE","DP05_0075PE"]

# Force non-numeric values to NA
fl_acs[pop_cols] = fl_acs[pop_cols].apply(
    pd.to_numeric,
    errors="coerce")

fl_acs

{'GEO_ID': 'Geography', 'NAME': 'Geographic Area Name', 'DP05_0001E': 'Estimate!!SEX AND AGE!!Total population', 'DP05_0001M': 'Margin of Error!!SEX AND AGE!!Total population', 'DP05_0002E': 'Estimate!!SEX AND AGE!!Total population!!Male', 'DP05_0002M': 'Margin of Error!!SEX AND AGE!!Total population!!Male', 'DP05_0003E': 'Estimate!!SEX AND AGE!!Total population!!Female', 'DP05_0003M': 'Margin of Error!!SEX AND AGE!!Total population!!Female', 'DP05_0004E': 'Estimate!!SEX AND AGE!!Total population!!Sex ratio (males per 100 females)', 'DP05_0004M': 'Margin of Error!!SEX AND AGE!!Total population!!Sex ratio (males per 100 females)', 'DP05_0005E': 'Estimate!!SEX AND AGE!!Total population!!Under 5 years', 'DP05_0005M': 'Margin of Error!!SEX AND AGE!!Total population!!Under 5 years', 'DP05_0006E': 'Estimate!!SEX AND AGE!!Total population!!5 to 9 years', 'DP05_0006M': 'Margin of Error!!SEX AND AGE!!Total population!!5 to 9 years', 'DP05_0007E': 'Estimate!!SEX AND AGE!!Total population!!10 to 

/var/folders/f1/_qnbjphx7mvb9p2c19t_s1540000gn/T/ipykernel_54307/1741720929.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fl_acs[pop_cols] = fl_acs[pop_cols].apply(


,GEO_ID,NAME,DP05_0001E,DP05_0037PE,DP05_0075PE
0,1400000US12001000201,Census Tract 2.01; Alachua County; Florida,5187,72.5,5187
1,1400000US12001000202,Census Tract 2.02; Alachua County; Florida,5897,73.5,5897
2,1400000US12001000301,Census Tract 3.01; Alachua County; Florida,3703,73.5,3703
3,1400000US12001000302,Census Tract 3.02; Alachua County; Florida,2500,60.3,2500
4,1400000US12001000400,Census Tract 4; Alachua County; Florida,5736,51.5,5736
...,...,...,...,...,...
5155,1400000US12133970104,Census Tract 9701.04; Washington County; Florida,2564,86.9,2564
5156,1400000US12133970200,Census Tract 9702; Washington County; Florida,3225,88.4,3225
5157,1400000US12133970301,Census Tract 9703.01; Washington County; Florida,2712,87.4,2712
5158,1400000US12133970302,Census Tract 9703.02; Washington County; Florida,7094,65.0,7094


### Prep Schools Data

In [4]:
# ---- Explore & prep schools data -----

# Explore data
public_schools.head
private_schools.head

# Add descriptive column for schools
public_schools["school_type"] = "public_schools"
private_schools["school_type"] = "private_schools"

# Join schools data
schools = pd.concat([public_schools, private_schools], ignore_index = True)

In [5]:
# --- Count schools in florida tracts ----

# Left join with florida tracts 
schools_in_florida = gpd.sjoin(
    florida_tracts,
    schools,
    how = "left",
    predicate = "intersects")


# Count schools per tract unique tract using GEOID
school_counts = (schools_in_florida
                 .groupby(["GEOIDFQ","school_type"])
                 .size()
                 .unstack(fill_value=0)
                 .reset_index())

### Merge Datasets

In [6]:
# ---- Merge acs data to florida tracts ----

# Left merge school counts on florida tracts
florida_tracts = florida_tracts.merge(
    fl_acs,
    how = "left",
    left_on = "GEOIDFQ",
    right_on = "GEO_ID")


In [7]:
# ---- Merge school data to florida tracts ---

# Left merge school counts on florida tracts
florida_tracts = florida_tracts.merge(
    school_counts,
    how = "left",
    on = "GEOIDFQ")

# Fill in NAs for no schools with 0s
florida_tracts[["public_schools", "private_schools"]] = (
    florida_tracts[["public_schools", "private_schools"]].fillna(0))

In [11]:
florida_tracts

,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME_x,NAMELSAD,MTFCC,FUNCSTAT,ALAND,...,DP05_0001E_x,DP05_0037PE_x,DP05_0075PE_x,GEO_ID_y,NAME,DP05_0001E_y,DP05_0037PE_y,DP05_0075PE_y,private_schools,public_schools
0,12,099,004300,12099004300,1400000US12099004300,43,Census Tract 43,G5020,S,4826118,...,6244.0,70.0,6244.0,1400000US12099004300,Census Tract 43; Palm Beach County; Florida,6244.0,70.0,6244.0,1.0,1.0
1,12,099,004500,12099004500,1400000US12099004500,45,Census Tract 45,G5020,S,2905159,...,7869.0,29.8,7869.0,1400000US12099004500,Census Tract 45; Palm Beach County; Florida,7869.0,29.8,7869.0,0.0,0.0
2,12,099,004602,12099004602,1400000US12099004602,46.02,Census Tract 46.02,G5020,S,1984760,...,5209.0,29.5,5209.0,1400000US12099004602,Census Tract 46.02; Palm Beach County; Florida,5209.0,29.5,5209.0,0.0,1.0
3,12,099,004702,12099004702,1400000US12099004702,47.02,Census Tract 47.02,G5020,S,2645473,...,7441.0,23.2,7441.0,1400000US12099004702,Census Tract 47.02; Palm Beach County; Florida,7441.0,23.2,7441.0,0.0,2.0
4,12,099,004704,12099004704,1400000US12099004704,47.04,Census Tract 47.04,G5020,S,2657506,...,7093.0,32.1,7093.0,1400000US12099004704,Census Tract 47.04; Palm Beach County; Florida,7093.0,32.1,7093.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5155,12,057,011805,12057011805,1400000US12057011805,118.05,Census Tract 118.05,G5020,S,1447981,...,3894.0,42.2,3894.0,1400000US12057011805,Census Tract 118.05; Hillsborough County; Florida,3894.0,42.2,3894.0,0.0,0.0
5156,12,057,011911,12057011911,1400000US12057011911,119.11,Census Tract 119.11,G5020,S,2077714,...,6139.0,42.9,6139.0,1400000US12057011911,Census Tract 119.11; Hillsborough County; Florida,6139.0,42.9,6139.0,0.0,0.0
5157,12,057,002702,12057002702,1400000US12057002702,27.02,Census Tract 27.02,G5020,S,1235845,...,2806.0,41.0,2806.0,1400000US12057002702,Census Tract 27.02; Hillsborough County; Florida,2806.0,41.0,2806.0,0.0,0.0
5158,12,091,023100,12091023100,1400000US12091023100,231,Census Tract 231,G5020,S,6557738,...,8609.0,60.1,8609.0,1400000US12091023100,Census Tract 231; Okaloosa County; Florida,8609.0,60.1,8609.0,0.0,1.0


### Calculate Spatial Statistics

In [8]:
# ---- Calculate school statistics ----

# Calculate total schools per tract
florida_tracts["total_schools"] = (florida_tracts["public_schools"] + florida_tracts["private_schools"])

# Calculate schools per capita
florida_tracts["schools_per_capita"] = (florida_tracts["total_schools"]/ florida_tracts["DP05_0001E"])

# Calculate schools per capita
florida_tracts["schools_per_1000"] = florida_tracts["schools_per_capita"] * 1000

In [9]:
florida_tracts

,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME_x,NAMELSAD,MTFCC,FUNCSTAT,ALAND,...,GEO_ID,NAME_y,DP05_0001E,DP05_0037PE,DP05_0075PE,private_schools,public_schools,total_schools,schools_per_capita,schools_per_1000
0,12,099,004300,12099004300,1400000US12099004300,43,Census Tract 43,G5020,S,4826118,...,1400000US12099004300,Census Tract 43; Palm Beach County; Florida,6244,70.0,6244,1.0,1.0,2.0,0.000320,0.320307
1,12,099,004500,12099004500,1400000US12099004500,45,Census Tract 45,G5020,S,2905159,...,1400000US12099004500,Census Tract 45; Palm Beach County; Florida,7869,29.8,7869,0.0,0.0,0.0,0.000000,0.000000
2,12,099,004602,12099004602,1400000US12099004602,46.02,Census Tract 46.02,G5020,S,1984760,...,1400000US12099004602,Census Tract 46.02; Palm Beach County; Florida,5209,29.5,5209,0.0,1.0,1.0,0.000192,0.191975
3,12,099,004702,12099004702,1400000US12099004702,47.02,Census Tract 47.02,G5020,S,2645473,...,1400000US12099004702,Census Tract 47.02; Palm Beach County; Florida,7441,23.2,7441,0.0,2.0,2.0,0.000269,0.268781
4,12,099,004704,12099004704,1400000US12099004704,47.04,Census Tract 47.04,G5020,S,2657506,...,1400000US12099004704,Census Tract 47.04; Palm Beach County; Florida,7093,32.1,7093,2.0,2.0,4.0,0.000564,0.563936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5155,12,057,011805,12057011805,1400000US12057011805,118.05,Census Tract 118.05,G5020,S,1447981,...,1400000US12057011805,Census Tract 118.05; Hillsborough County; Florida,3894,42.2,3894,0.0,0.0,0.0,0.000000,0.000000
5156,12,057,011911,12057011911,1400000US12057011911,119.11,Census Tract 119.11,G5020,S,2077714,...,1400000US12057011911,Census Tract 119.11; Hillsborough County; Florida,6139,42.9,6139,0.0,0.0,0.0,0.000000,0.000000
5157,12,057,002702,12057002702,1400000US12057002702,27.02,Census Tract 27.02,G5020,S,1235845,...,1400000US12057002702,Census Tract 27.02; Hillsborough County; Florida,2806,41.0,2806,0.0,0.0,0.0,0.000000,0.000000
5158,12,091,023100,12091023100,1400000US12091023100,231,Census Tract 231,G5020,S,6557738,...,1400000US12091023100,Census Tract 231; Okaloosa County; Florida,8609,60.1,8609,0.0,1.0,1.0,0.000116,0.116158
